Code written by Nasim Khadem

### Question

1. It uses the sample dataset https://raw.githubusercontent.com/localytics/data-viz-challenge/master/data.json

2. Loads this data (make sure that you don't manually download the data but do it programmatically), processes it and outputs the following CSV file to s3:

3. Create an output file  total_events with these parameters: 

a. it has the following columns:  age, device, date, count, sum.

b. group on gender, age, device and date of client_time 

c. the date column has the format of YYYY-MM-DD

d. count column is the count of entries

e. sum column is the summing the value of the 'amount' key

f. only calculates entries for female ( "gender": "F" ) and Californian ("state": "CA") users.

You can use Python libraries if you wish. For completing the assignment please send me the program that you wrote, and the location of the CSV output file.



### Modules
Importing Python Modules

In [1]:
import pandas as pd 
import os
import numpy as np
import json,urllib.request
import datetime
import boto3


### Load
Reading the json file programatically from the given URL.

In [25]:
url="https://raw.githubusercontent.com/localytics/data-viz-challenge/master/data.json"
data = urllib.request.urlopen(url).read()
output = json.loads(data)

Write the json to file for easy acess.

In [3]:
json_file_name=os.path.join(url.split ("/")[-1])
with open(json_file_name, mode='w') as outfile:
    json.dump(output, outfile)

Reading the json file into a dataframe 

In [4]:
df = pd.read_json(json_file_name,orient='split')

Inspecting the dataframe to have a sense of the data.

In [5]:
df.head(5)

,age,amount,category,client_time,device,event_name,gender,location,marital_status,session_id
0,18-24,NaN,Sports,2014-03-01 00:00:04,android,View Project,M,"{'latitude': 40.189788, 'city': 'Lyons', 'stat...",married,69f62d2ae87640f5a2dde2b2e9229fe6
1,18-24,NaN,Technology,2014-03-01 00:00:22,iOS,View Project,M,"{'latitude': 33.844371, 'city': 'Alpharetta', ...",single,4459d001feb8438eae5f4ec24abcd992
2,55+,NaN,Environment,2014-03-01 00:00:32,iOS,View Project,M,"{'latitude': 42.446396, 'city': 'Westford', 's...",single,0db9ed700a184d48a9d04806696e3642
3,18-24,NaN,Technology,2014-03-01 00:00:38,android,View Project,M,"{'latitude': 44.624413, 'city': 'Saranac', 'st...",single,68195e2372bd4022b17220fc21de9138
4,25-34,NaN,Sports,2014-03-01 00:00:51,iOS,View Project,F,"{'latitude': 36.747083, 'city': 'Lampe', 'stat...",married,9508a8385dc94773baba8aa7d1c2aa75


### location
The location is not in the correct format. 

In [6]:
df=pd.concat([df.drop(['location'], axis=1), df['location'].apply(pd.Series)], axis=1)

In [7]:
df.head(5)

,age,amount,category,client_time,device,event_name,gender,marital_status,session_id,city,latitude,longitude,state,zip_code
0,18-24,NaN,Sports,2014-03-01 00:00:04,android,View Project,M,married,69f62d2ae87640f5a2dde2b2e9229fe6,Lyons,40.189788,-105.355280,CO,80540
1,18-24,NaN,Technology,2014-03-01 00:00:22,iOS,View Project,M,single,4459d001feb8438eae5f4ec24abcd992,Alpharetta,33.844371,-84.474050,GA,30009
2,55+,NaN,Environment,2014-03-01 00:00:32,iOS,View Project,M,single,0db9ed700a184d48a9d04806696e3642,Westford,42.446396,-71.459405,MA,01886
3,18-24,NaN,Technology,2014-03-01 00:00:38,android,View Project,M,single,68195e2372bd4022b17220fc21de9138,Saranac,44.624413,-73.809266,NY,12981
4,25-34,NaN,Sports,2014-03-01 00:00:51,iOS,View Project,F,married,9508a8385dc94773baba8aa7d1c2aa75,Lampe,36.747083,-93.458626,MO,65681


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50000 entries, 0 to 49999
Data columns (total 14 columns):
age               50000 non-null object
amount            20658 non-null float64
category          50000 non-null object
client_time       50000 non-null datetime64[ns]
device            50000 non-null object
event_name        50000 non-null object
gender            50000 non-null object
marital_status    50000 non-null object
session_id        50000 non-null object
city              50000 non-null object
latitude          50000 non-null float64
longitude         50000 non-null float64
state             50000 non-null object
zip_code          50000 non-null object
dtypes: datetime64[ns](1), float64(3), object(10)
memory usage: 5.7+ MB


### date format
client date has to be changed to " the date column has the format of YYYY-MM-DD"

In [9]:
pd.to_datetime(df['client_time'], format='%Y%m%d', errors='coerce')
df['date'] = df['client_time'].dt.date

The amount has many nans , here I fill  NaNs with zeros.

In [10]:
df['amount'].fillna(0, inplace=True)

check to see if the date and Nans have been replaced correctly

In [11]:
df.tail()

,age,amount,category,client_time,device,event_name,gender,marital_status,session_id,city,latitude,longitude,state,zip_code,date
49995,18-24,35.0,Sports,2014-03-31 23:48:40,iOS,Fund Project,F,married,412b973788704c36a008a506cdbba033,West Linn,45.343615,-122.664926,OR,97068,2014-03-31
49996,45-54,0.0,Technology,2014-03-31 23:49:29,android,View Project,F,single,4addd36a6f4347c59865fe04a92e8d57,Greenwich,41.079983,-73.654472,CT,06831,2014-03-31
49997,25-34,0.0,Fashion,2014-03-31 23:51:33,iOS,View Project,F,single,3181a138a0b94d1da9a3d29f7816fcc7,Irwin,40.365020,-79.584060,PA,15642,2014-03-31
49998,35-44,37.0,Sports,2014-03-31 23:52:48,iOS,Fund Project,F,married,2df6b9a0c66b48a389330327517b9276,Martinsville,39.477625,-86.466792,IN,46151,2014-03-31
49999,18-24,0.0,Fashion,2014-03-31 23:58:45,iOS,View Project,M,single,fc24339bfda84ff29102981cbee78023,Lake Oswego,45.408374,-122.691388,OR,97034,2014-03-31


### Filter by State and gender
Filter out the data for "only calculates entries for female ( "gender": "F" ) and Californian ("state": "CA") users."

In [12]:
f_df=df[(df['state']=='CA') & (df['gender']=='F')]

In [13]:
f_df

,age,amount,category,client_time,device,event_name,gender,marital_status,session_id,city,latitude,longitude,state,zip_code,date
7,35-44,0.0,Technology,2014-03-01 00:02:18,android,View Project,F,married,c6f18d84b43b4a4a90fa9a44016c3665,Los Banos,37.103768,-120.847479,CA,93635,2014-03-01
68,55+,0.0,Fashion,2014-03-01 00:15:38,android,View Project,F,married,4e7beaf4ca094f75a0fd06004744607b,Modesto,37.669463,-121.016796,CA,95350,2014-03-01
71,35-44,33.0,Technology,2014-03-01 00:17:18,android,Fund Project,F,married,c6f18d84b43b4a4a90fa9a44016c3665,Los Banos,37.103768,-120.847479,CA,93635,2014-03-01
72,35-44,0.0,Fashion,2014-03-01 00:17:28,android,View Project,F,married,c6f18d84b43b4a4a90fa9a44016c3665,Los Banos,37.103768,-120.847479,CA,93635,2014-03-01
90,45-54,0.0,Sports,2014-03-01 00:22:24,android,View Project,F,single,0c0aadfe4f0d482da515eb32e5659d6d,Desert Hot Springs,33.900103,-116.400701,CA,92240,2014-03-01
97,18-24,0.0,Environment,2014-03-01 00:23:52,android,View Project,F,married,f6b6a562d4634a5d80fc743642df5ab0,Chula Vista,32.615983,-117.034012,CA,91911,2014-03-01
179,25-34,0.0,Sports,2014-03-01 00:41:34,android,View Project,F,single,b3241de09a664fe3a9230feb17b42d27,Ontario,34.839964,-115.967051,CA,91764,2014-03-01
255,35-44,0.0,Fashion,2014-03-01 01:01:07,iOS,View Project,F,married,7541fbf3bc5f4249b091b118e853fd5b,Chino Hills,34.839964,-115.967051,CA,91709,2014-03-01
353,25-34,0.0,Sports,2014-03-01 01:24:39,iOS,View Project,F,married,cb9da8af120b4c85ab29e74bf6d8b382,Moreno Valley,33.857218,-116.720427,CA,92553,2014-03-01
494,18-24,0.0,Sports,2014-03-01 02:07:01,iOS,View Project,F,single,5fd22dafbdeb4995ad0d094fe5aebb92,Canoga Park,33.786594,-118.298662,CA,91303,2014-03-01


### Aggregate
Groupby the data and create the sum and count of each index. the count can be taken on any columns, here i take gender

In [14]:
total_amount=f_df.groupby(['age', 'gender','device','date']).agg({'amount' : 'sum','gender':'count'})

In [15]:
total_amount

amount  gender
age   gender device  date                      
18-24 F      android 2014-03-01    31.0      13
                     2014-03-02    69.0       5
                     2014-03-03    40.0       6
                     2014-03-04    49.0       7
                     2014-03-05   113.0       5
                     2014-03-06     1.0       2
                     2014-03-07   116.0       8
                     2014-03-08     0.0       9
                     2014-03-09    27.0       1
                     2014-03-10   105.0       9
                     2014-03-11    45.0       4
                     2014-03-12   200.0       6
                     2014-03-13   142.0      11
                     2014-03-14    85.0       4
                     2014-03-15   112.0       3
                     2014-03-16     0.0       3
                     2014-03-17    57.0       4
                     2014-03-18    48.0       5
                     2014-03-19    93.0       5
                     2014-03-20   177.0       5
                     2014-03-21    67.0       3
                     2014-03-22    88.0       7
                     2014-03-23    57.0       1
                     2014-03-24   145.0       5
                     2014-03-25    48.0       2
                     2014-03-26    76.0       5
                     2014-03-27     0.0       1
                     2014-03-28    99.0       4
                     2014-03-29    54.0       4
                     2014-03-30   130.0       6
...                                 ...     ...
55+   F      iOS     2014-03-01     9.0      10
                     2014-03-02    83.0       7
                     2014-03-03    29.0       5
                     2014-03-04     0.0       4
                     2014-03-05    82.0       6
                     2014-03-06     0.0       1
                     2014-03-07     0.0       3
                     2014-03-08   106.0       5
                     2014-03-09    32.0       7
                     2014-03-10    27.0       5
                     2014-03-11   131.0       2
                     2014-03-12    58.0       2
                     2014-03-13     0.0       2
                     2014-03-14     0.0       3
                     2014-03-15    83.0       6
                     2014-03-16    35.0       2
                     2014-03-17     0.0       3
                     2014-03-18   112.0       5
                     2014-03-19    50.0       1
                     2014-03-20   141.0       5
                     2014-03-21    40.0       3
                     2014-03-22     0.0       2
                     2014-03-23    49.0       2
                     2014-03-24    29.0       1
                     2014-03-25   104.0       3
                     2014-03-26   164.0       5
                     2014-03-27     0.0       2
                     2014-03-28    60.0       2
                     2014-03-29     0.0       2
                     2014-03-30    86.0       3

[300 rows x 2 columns]

Rename the amount and count of gender to match the file 

In [16]:
total_amount.rename(index=str, columns={"amount": "sum", "gender": "count"},inplace=True)

In [17]:
total_amount.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 300 entries, (18-24, F, android, 2014-03-01) to (55+, F, iOS, 2014-03-30)
Data columns (total 2 columns):
sum      300 non-null float64
count    300 non-null int64
dtypes: float64(1), int64(1)
memory usage: 6.4+ KB


reset the index 

In [18]:
total_amount.reset_index(inplace=True)

In [19]:
total_amount

,age,gender,device,date,sum,count
0,18-24,F,android,2014-03-01,31.0,13
1,18-24,F,android,2014-03-02,69.0,5
2,18-24,F,android,2014-03-03,40.0,6
3,18-24,F,android,2014-03-04,49.0,7
4,18-24,F,android,2014-03-05,113.0,5
5,18-24,F,android,2014-03-06,1.0,2
6,18-24,F,android,2014-03-07,116.0,8
7,18-24,F,android,2014-03-08,0.0,9
8,18-24,F,android,2014-03-09,27.0,1
9,18-24,F,android,2014-03-10,105.0,9


### Tiny test 
Test a random row see if it is working correctly

18-24 	F 	android 	2014-03-10 	105.0 	9

In [24]:
df[(df['age']=='18-24') & (df['device']=='android') &
   (df['date']==datetime.date(2014,3,10)) & (df['state']=='CA') & (df['gender']=='F')].sum()

amount        105.000000
latitude      315.520548
longitude   -1066.170009
dtype: float64

In [20]:
df[(df['age']=='18-24') & (df['device']=='android') &
   (df['date']==datetime.date(2014,3,10)) & (df['state']=='CA') & (df['gender']=='F')].describe()

,amount,latitude,longitude
count,9.000000,9.000000,9.000000
mean,11.666667,35.057839,-118.463334
std,24.748737,2.089426,2.173986
min,0.000000,32.775849,-122.124225
25%,0.000000,33.640302,-119.592146
50%,0.000000,33.786594,-117.769442
75%,0.000000,36.832583,-117.207965
max,70.000000,38.547639,-115.967051


### output to csv 
Write the total_amount to a CSV file 

In [21]:
total_amount.to_csv('total_amount.csv')

### S3 Upload
write to s3. First the credential was configured using  AWS CLI  and then the file was uploaded. The code was taken from aws help file.I created the bucket manually could be done also using code. I added the code as the comment 

In [22]:
# Copyright 2010-2019 Amazon.com, Inc. or its affiliates. All Rights Reserved.
#
# This file is licensed under the Apache License, Version 2.0 (the "License").
# You may not use this file except in compliance with the License. A copy of the
# License is located at
#
# http://aws.amazon.com/apache2.0/
#
# This file is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS
# OF ANY KIND, either express or implied. See the License for the specific
# language governing permissions and limitations under the License.





# Create an S3 client
s3 = boto3.client('s3')

filename = 'total_amount.csv'
bucket_name = 'ecosiatest'

# Create bucket 
#s3.create_bucket(Bucket=bucket_name)

# Uploads the given file using a managed uploader, which will split up large
# files automatically and upload parts in parallel.
s3.upload_file(filename, bucket_name, filename)
 

### Download
To download the total_amount file go to :
    https://s3.eu-central-1.amazonaws.com/ecosiatest/total_amount.csv

#### challenges
- The aws was completely new for me , had to read a bit on it. 
- The aggregation part was kind of fun. 
- Using pandas made everything easy.
- I first did  part f since it made the aggregation part faster. 
- Stackoverflow and documentations are a gals best friend.
- Over all the most important thing was that I had fun working on this.
